In [1]:
import numpy as np
import skimage
from skimage.util.shape import view_as_blocks
import os
import shutil
import json

In [2]:
image_path = "/home/gauthamar11/pyLattice2/src/tensorflow/quickUnet/dataset/extras/S3P5_488_150mw_560_300mw_Objdz150nm_ch1_CAM1_stack0001_560nm_0005689msec_0090121790msecAbs_000x_000y_003z_0000t_decon.tif"
mask_path= "/home/gauthamar11/pyLattice2/src/tensorflow/quickUnet/dataset/extras/dmask_02.tif"
split_directory="/home/gauthamar11/pyLattice2/src/tensorflow/quickUnet/genData/"
patch_size = 96
train_split = 1 #Trying to get coverage of whole large dataset frame. Can change once we use more frames of our large data

if "train" not in os.listdir(split_directory):
    os.mkdir(split_directory+"train/")
if "test" not in os.listdir(split_directory):
    os.mkdir(split_directory+"test/")

In [5]:
latticeMovieImage = skimage.external.tifffile.imread(image_path)
latticeMovieMask = skimage.external.tifffile.imread(mask_path)
offset=np.asarray([0,0,0])

x_extra = latticeMovieImage.shape[0]%patch_size
x_size = latticeMovieImage.shape[0] - x_extra
if offset[0] > x_extra:
    print("1st dim offset exceeds image dim")
    offset[0] = 0
    
y_extra = latticeMovieImage.shape[1]%patch_size
y_size = latticeMovieImage.shape[1] - y_extra
if offset[1] > y_extra:
    print("2st dim offset exceeds image dim")
    offset[1] = 0
    
z_extra = latticeMovieImage.shape[2]%patch_size
z_size = latticeMovieImage.shape[2] - z_extra
if offset[2] > z_extra:
    print("3rd dim offset exceeds image dim")
    offset[2] = 0
    
latticeMovieImage = latticeMovieImage[offset[0]:x_size+offset[0], offset[1]:y_size+offset[1], offset[2]:z_size+offset[2]]
latticeMovieMask = latticeMovieMask[offset[0]:x_size+offset[0], offset[1]:y_size+offset[1], offset[2]:z_size+offset[2]]
print("Image cropped to: " + str(x_size) + ", " + str(y_size) + ", " + str(z_size))

print(latticeMovieImage.shape)
print(latticeMovieMask.shape)
print(np.amax(latticeMovieMask))

Image cropped to: 192, 672, 576
(192, 672, 576)
(192, 672, 576)
255


In [5]:
lattice_patches = view_as_blocks(latticeMovieImage, block_shape=(patch_size, patch_size, patch_size))
lattice_patches = lattice_patches.reshape(int(x_size/patch_size)*int(y_size/patch_size)*int(z_size/patch_size), patch_size, patch_size, patch_size)


mask_patches = view_as_blocks(latticeMovieMask, block_shape=(patch_size, patch_size, patch_size))
mask_patches = mask_patches.reshape(int(x_size/patch_size)*int(y_size/patch_size)*int(z_size/patch_size), patch_size, patch_size, patch_size)


print(lattice_patches.shape)
print(mask_patches.shape)

/usr/local/lib/python3.5/dist-packages/skimage/util/shape.py:94: RuntimeWarning: Cannot provide views on a non-contiguous input array without copying.
  warn(RuntimeWarning("Cannot provide views on a non-contiguous input "


(84, 96, 96, 96)
(84, 96, 96, 96)


In [7]:
num_patches = lattice_patches.shape[0]

for k in range(0, num_patches):
    x_file = lattice_patches[k].astype('uint16')
    y_file = mask_patches[k].astype('uint16')
    
    metadata_x = dict(microscope='joh', shape=x_file.shape, dtype=x_file.dtype.str)
    metadata_x = json.dumps(metadata_x)
    
    metadata_y = dict(microscope='joh', shape=y_file.shape, dtype=y_file.dtype.str)
    metadata_y = json.dumps(metadata_y)
    
    os.mkdir(split_directory+"train/Region"+str(k)+"/")
    skimage.external.tifffile.imsave(split_directory+"train/Region"+str(k)+"/"+"lattice_light_sheet.tif", x_file, description=metadata_x)
    skimage.external.tifffile.imsave(split_directory+"train/Region"+str(k)+"/"+"truth.tif", y_file, description=metadata_y)
    